In [2]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

In [3]:
train_client=pd.read_csv('r:/train/client_train.csv')
train_invoice=pd.read_csv('r:/train/invoice_train.csv')
test_client=pd.read_csv('r:/train/client_test.csv')
test_invoice=pd.read_csv('r:/train/invoice_test.csv')
sub=pd.read_csv('r:/train/SampleSubmission.csv')

In [4]:
for df in [train_invoice,test_invoice]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    df['year'] = df['invoice_date'].dt.year
    df['month'] = df['invoice_date'].dt.month

In [5]:
for df in [train_client,test_client]:
    df['creation_date'] = pd.to_datetime(df['creation_date'])
    df['year'] = df['creation_date'].dt.year
    df['month'] = df['creation_date'].dt.month

In [6]:
d={"ELEC":0,"GAZ":1}
train_invoice['counter_type']=train_invoice['counter_type'].map(d)
d={"ELEC":0,"GAZ":1}
test_invoice['counter_type']=test_invoice['counter_type'].map(d)

In [68]:
aggs = {}
aggs['consommation_level_1'] = ['sum','max','min','mean']
aggs['consommation_level_2'] = ['sum','max','min','mean']
aggs['consommation_level_3'] = ['sum','max','min','mean']
aggs['consommation_level_4'] = ['sum','max','min','mean']

aggs['month'] = ['mean', 'max', 'min']
aggs['year'] = ['nunique','max','min','mean']

aggs['months_number'] = ['max','min','mean','sum']
aggs['reading_remarque'] = ['max','min','mean','sum']
aggs['counter_coefficient'] = ['max', 'min','mean','sum']
aggs['counter_number'] = ['nunique','max','min']
aggs['counter_type'] = ['nunique','mean','sum']
aggs['counter_statue'] = ['nunique','max','min','sum']
aggs['tarif_type'] = ['nunique','max','min','sum']
aggs['counter_code'] = ['nunique','max','mean','min']


aggs['old_index'] = ['nunique','mean','max','min']
aggs['new_index'] = ['nunique','mean','max','min']

In [69]:
agg_train = train_invoice.groupby(['client_id']).agg(aggs)
agg_test = test_invoice.groupby(['client_id']).agg(aggs)


In [70]:
agg_train.columns = ['_'.join(col).strip() for col in agg_train.columns.values]
agg_test.columns = ['_'.join(col).strip() for col in agg_test.columns.values]

In [71]:
train = pd.merge(train_client,agg_train, on='client_id', how='left')
test = pd.merge(test_client,agg_test, on='client_id', how='left')


In [72]:
target=train['target']


In [73]:
train = train.drop(['target','creation_date'], axis=1)


In [74]:
test = test.drop(['creation_date'], axis=1)

In [78]:
from sklearn import preprocessing
for f in train.columns:
    if train[f].dtype=='object' : 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values))
        train[f] = lbl.transform(list(train[f].values))

In [79]:
from sklearn import preprocessing
for f in test.columns:
    if test[f].dtype=='object' : 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test[f].values))
        test[f] = lbl.transform(list(test[f].values))
        

# xgb classement 2


In [80]:
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=2019,
)
%time clf.fit(train, target)

Wall time: 1min 47s


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=2019, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.9, tree_method=None,
              validate_parameters=False, verbosity=None)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3)

In [29]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=2019, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.9, tree_method=None,
              validate_parameters=False, verbosity=None)

In [81]:
y_pred = clf.predict_proba(test) 
print(y_pred)

[[0.97968215 0.02031787]
 [0.7505926  0.24940738]
 [0.9851574  0.01484265]
 ...
 [0.2080071  0.7919929 ]
 [0.9969078  0.00309224]
 [0.9261943  0.0738057 ]]


In [31]:
from sklearn.metrics import accuracy_score
# Evaluate accuracy
print(accuracy_score(y_test, y_pred))

0.947156071639441


In [82]:
pred = pd.DataFrame(y_pred)
print(pred)

              0         1
0      0.979682  0.020318
1      0.750593  0.249407
2      0.985157  0.014843
3      0.996289  0.003711
4      0.961391  0.038609
...         ...       ...
58064  0.999268  0.000732
58065  0.990326  0.009674
58066  0.208007  0.791993
58067  0.996908  0.003092
58068  0.926194  0.073806

[58069 rows x 2 columns]


In [83]:
submission = pd.DataFrame({
        "client_id": sub["client_id"],
        "target": pred[1]
    })
submission.to_csv('r:steg67.csv', index=False)